In [16]:
import numpy as np
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df = pl.read_csv("../../DATA/Advertising.csv")

In [18]:
df

TV,radio,newspaper,sales
f64,f64,f64,f64
230.1,37.8,69.2,22.1
44.5,39.3,45.1,10.4
17.2,45.9,69.3,9.3
151.5,41.3,58.5,18.5
180.8,10.8,58.4,12.9
…,…,…,…
38.2,3.7,13.8,7.6
94.2,4.9,8.1,9.7
177.0,9.3,6.4,12.8


In [19]:
X = df.drop(["sales"])
y = df["sales"]

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=101)

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

In [24]:
scaler.fit(X_train)

StandardScaler()

In [25]:
scaled_X_train = scaler.fit_transform(X_train)

In [26]:
scaled_X_test = scaler.transform(X_test)

In [27]:
from sklearn.linear_model import Ridge

In [28]:
ridge_model = Ridge(alpha=100)

In [29]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=100)

In [30]:
y_predict = ridge_model.predict(X_test)

In [35]:
predict_df = pl.DataFrame({
    "y_test": y_test,
    "y_predict": y_predict,
    "error": y_test - y_predict,
    'error %': (y_test - y_predict) / y_test * 100
})

In [36]:
predict_df

y_test,y_predict,error,error %
f64,f64,f64,f64
14.7,15.725804,-1.025804,-6.978258
19.8,19.604452,0.195548,0.987615
11.9,11.454096,0.445904,3.747094
16.7,16.994599,-0.294599,-1.764068
9.5,9.167855,0.332145,3.496258
…,…,…,…
3.2,5.513072,-2.313072,-72.283496
25.4,23.276901,2.123099,8.358659
10.8,12.608394,-1.808394,-16.744386


In [39]:
abs(predict_df['error %']).mean()

11.82462436542779